In [8]:
import json
import pymongo

In [9]:
from data_tool import flat_json

In [3]:
# db set-up
client = pymongo.MongoClient("localhost", 27017)
db = client.fanpage_research
db.name

u'fanpage_research'

In [4]:
def get_valid_pid_list():
    """
    Get all post_id from MongoDB excluding the post which content is None and like_count is 0.

    :return: valid post for research.
    :rtype: list
    """
    valid_pid_list = []
    try:
        # get pid_list exclude message content is None
        p_cursor = list(db.post.aggregate([{"$match":{"message":{"$ne":None}}}]))
        pid_list_has_message = flat_json(p_cursor)['pid']

        # get pid_list exclude like_count is 0
        pd_cursor = list(db.post_detail.aggregate([
            {"$group":{"_id":"$pid","like_count_total":{"$sum":"$like_count"}}},
            {"$match":{"like_count_total":{"$ne":0}}}
        ]))
        pid_list_has_like = flat_json(pd_cursor)['_id']

        # get valid_pid_list where pid has message content and like_count is not 0
        valid_pid_list = list(set.intersection(set(pid_list_has_message),set(pid_list_has_like)))
    except:
        print "Error occured when getting valid_pid_list"
    return valid_pid_list

In [5]:
valid_pid_list = get_valid_pid_list()

In [6]:
# 將 valid post_list 存入硬碟中，待後續使用
with open("data_106/valid_pid_list",'w') as outfile:
    json.dump(valid_pid_list,outfile)